In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/UnderSampledDataUsingRepeatedEnn/undersampled_using_repeated_enn.csv


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/UnderSampledDataUsingRepeatedEnn/undersampled_using_repeated_enn.csv')

# **Using Simple LSTM**

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import np_utils


# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape the data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Epoch 1/5
2775/2775 [==============================] - 37s 11ms/step - loss: 0.3209 - accuracy: 0.8852 - val_loss: 0.1922 - val_accuracy: 0.9243
Epoch 2/5
2775/2775 [==============================] - 30s 11ms/step - loss: 0.2000 - accuracy: 0.9237 - val_loss: 0.1742 - val_accuracy: 0.9326
Epoch 3/5
2775/2775 [==============================] - 30s 11ms/step - loss: 0.1840 - accuracy: 0.9296 - val_loss: 0.1860 - val_accuracy: 0.9299
Epoch 4/5
2775/2775 [==============================] - 30s 11ms/step - loss: 0.1769 - accuracy: 0.9311 - val_loss: 0.1745 - val_accuracy: 0.9317
Epoch 5/5
2775/2775 [==============================] - 30s 11ms/step - loss: 0.1695 - accuracy: 0.9340 - val_loss: 0.1646 - val_accuracy: 0.9355
Test loss: 0.16461579501628876
Test accuracy: 0.9355107545852661


# **Use Optuna to Optimize Hyper Parameter LSTM**

In [4]:
!pip install optuna
!pip install -q keras

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import np_utils
import optuna
from keras import optimizers


# Define the objective function to be optimized
def objective(trial):
    # Define the hyperparameters to be tuned
    n_layers = trial.suggest_int("n_layers", 1, 3)
    n_units = trial.suggest_int("n_units", 50, 200)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.5)
    learning_rate = trial.suggest_uniform("learning_rate", 1e-5, 1e-2)

    # Extract the features and label
    features = df.drop("Label", axis=1).values
    labels = df["Label"].values

    # Encode the labels to numeric values
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)
    labels = np_utils.to_categorical(labels)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Reshape the data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Define the LSTM model
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            model.add(LSTM(n_units, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
        elif i == n_layers - 1:
            model.add(LSTM(n_units))
        else:
            model.add(LSTM(n_units, return_sequences=True))
        model.add(Dropout(dropout_rate))

    model.add(Dense(labels.shape[1], activation="softmax"))

    # Compile the model with the learning rate
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=0)

    # Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)

    return score[1]  # Return the validation accuracy as the objective value

# Define the study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Run 100 trials to optimize the hyperparameters

# Print the best hyperparameters and the corresponding accuracy
best_trial = study.best_trial
print("Best trial:")
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-02-22 17:50:30,598] A new study created in memory with name: no-name-dc3a7fc2-903f-406b-a937-bdad7b006409
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the autoencoder model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(features.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(features.shape[1], activation='sigmoid'))

# Compile the autoencoder model
model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder model
model.fit(X_train, X_train, epochs=50, batch_size=64, shuffle=True, validation_data=(X_test, X_test))

# Extract the encoded features from the autoencoder model
encoder_model = Sequential()
encoder_model.add(model.layers[0])
encoder_model.add(model.layers[1])
encoder_model.add(model.layers[2])
encoded_train = encoder_model.predict(X_train)
encoded_test = encoder_model.predict(X_test)

# Define the LSTM model with encoded features
lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(encoded_train.shape[1], encoded_train.shape[2]), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(labels.shape[1], activation="softmax"))

# Compile the LSTM model
lstm_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the LSTM model with encoded features
lstm_model.fit(encoded_train, y_train, epochs=5, batch_size=64, validation_data=(encoded_test, y_test))

# Evaluate the LSTM model with encoded features
score = lstm_model.evaluate(encoded_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Epoch 1/50
2775/2775 [==============================] - 11s 3ms/step - loss: nan - val_loss: nan
Epoch 2/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 3/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 4/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 5/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 6/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 7/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 8/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 9/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 10/50
2775/2775 [==============================] - 9s 3ms/step - loss: nan - val_loss: nan
Epoch 11/50
2775/2775 [===============

NameError: name 'LSTM' is not defined